In [ ]:
ens = '001'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
df_sub = pd.read_csv('/workspace/data/sample_submission.csv')
df = pd.read_csv('/workspace/data/df_train.csv')

In [ ]:
df_oof_list = [
    pd.read_csv('../exp/exp001/oof.csv'),
    pd.read_csv('../exp/exp002/oof.csv'),
    pd.read_csv('../exp/exp004/oof.csv'),
    pd.read_csv('../exp/exp009/oof.csv'),
    pd.read_csv('../exp/exp011/oof.csv'),
]



df_sub_list = [
    pd.read_csv('../exp/exp001/submit.csv'),
    pd.read_csv('../exp/exp002/submit.csv'),
    pd.read_csv('../exp/exp004/submit.csv'),
    pd.read_csv('../exp/exp009/submit.csv'),
    pd.read_csv('../exp/exp011/submit.csv'),
]

oof = np.array([o.pred for o in df_oof_list]).mean(0)
sub = np.array([o.target for o in df_sub_list]).mean(0)

human_index = df[df.label_type == "human"].index
auto_index = df[df.label_type == "auto"].index
print("Ensemble Oof CV score", roc_auc_score(df.target, oof))
print("Ensemble human Oof CV score", roc_auc_score(df.target.values[human_index], oof[human_index]))
print("Ensemble auto Oof CV score", roc_auc_score(df.target.values[auto_index], oof[auto_index]))

print()
cv_score = []
for i in range(5):
    cv_index = df[df.cv == i].index
    print(f"Ensemble Oof CV {i} score", roc_auc_score(df.target.values[cv_index], oof[cv_index]))
    cv_score.append(roc_auc_score(df.target.values[cv_index], oof[cv_index]))
print("mean of cv: ", np.mean(cv_score))
print()
    
print()
for th in [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]:
    print('----------')
    print(f"Threshold: {th}, acc {(df.target.values[oof>th]>th).sum()/(oof>th).sum():.3f}, total true {(oof>th).sum()}")
    print(f"Threshold: {th}, human acc {(df.target.values[human_index][oof[human_index]>th]>th).sum()/(oof[human_index]>th).sum():.3f}, total true {(oof[human_index]>th).sum()}")    
    print(f"Threshold: {th}, auto acc {(df.target.values[auto_index][oof[auto_index]>th]>th).sum()/(oof[auto_index]>th).sum():.3f}, total true {(oof[auto_index]>th).sum()}")
    
for th in [0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.01]:
    print('----------')
    print(f"Threshold: {th}, acc {(df.target.values[oof<th]<th).sum()/(oof<th).sum():.3f}, total true {(oof<th).sum()}")
    print(f"Threshold: {th}, human acc {(df.target.values[human_index][oof[human_index]<th]<th).sum()/(oof[human_index]<th).sum():.3f}, total true {(oof[human_index]<th).sum()}")    
    print(f"Threshold: {th}, auto acc {(df.target.values[auto_index][oof[auto_index]<th]<th).sum()/(oof[auto_index]<th).sum():.3f}, total true {(oof[auto_index]<th).sum()}")    

In [ ]:
df_sub["target"] = sub
df_sub.to_csv(f'./sub/ens{ens}.csv', index=False)